# Modeling the effects of GW pumping on surface bodies

In [4]:
#importing the environment
import flopy
import numpy as np
import matplotlib as mp
import os

#additional analysis tools
import flopy.utils.binaryfile as bf
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import csv

#jupyter specific--included to show plots in notebook
%matplotlib inline 

##LOOK HERE (GROUP 1)
dist_list = [2, 5 ,8 ,11 ,14]
count = -1
for i in dist_list:
    modelname = 'moldel' + '_' + str(i)
    moddir="C:\WRDAPP\MODFLOW\mf2005"
    m = flopy.modflow.Modflow(modelname, exe_name = moddir)
    #moddir = os.getcwd()+"\\modflowdir\\mf2005.exe"

    nrow = 20 #number of rows
    ncol = 15 #number of columns
    nlay = 1  #number of layers

    dx= 100 #width of grid cells in x diretion 
    dy= 100 #width of grid cells in y diretion 

    Lx = ncol*dx  #width of domain in x
    Ly = nrow*dy #width of domain in y
    ztop = 100.    #top elevation 
    zbot = 0.     #bottom elevation

    dz = (ztop - zbot) / nlay #cell height in z direction

    nper = 1 #number of stress periods 
    steady = [True] #list noting the stress period type

    #create flopy discretization object, length and time are meters (2) and days (4)
    dis = flopy.modflow.ModflowDis(model=m, nlay=nlay, nrow=nrow, ncol=ncol, 
                                   delr=dx, delc=dy, top=ztop, botm=zbot, 
                                   itmuni = 4, lenuni = 2, 
                                   nper=nper, steady=steady)



    #create ibound as array of ints = 1
    ibound = np.ones((nlay, nrow, ncol), dtype=np.int32) #integer array of dim (z,y,x), makes all cells active                                                     
    #set constand head boundary on the left
    ibound[:,:,0] = -1          #replace leftmost column (all zs, all ys, first x) with -1 to indicate constant head 

    #print("ibound values: \n", ibound)


    #setup initial heads as 1 everywhere and 100 on the left boundary 
    H_init = np.ones((nlay, nrow, ncol), dtype=np.float32)  #float array (i.e. decimal), sets all heads to 1.0
    H_init[:, :, 0] = 100    #replace first (left) col with desired head value

    #print("starting head values: \n", H_init)

    #create flopy bas object
    bas = flopy.modflow.ModflowBas(m, ibound=ibound, strt=H_init)


    K_horiz = 10.    #horizontal hydraulic conductivity 
    K_vert = 10    #assign vertical hydraulic conductivity (along z axis)
    n = 0.01        #assign porosity
    Ss = 0.01      #assign storage coefficient
    Sy = 0.01        #assign specific yield
    uncnf = 0      #0=confined, >0 = unconfined


    #assigns horiz  and vertical Ks and saves cell-by-cell budget data
    lpf = flopy.modflow.ModflowLpf(m, laytyp=uncnf, hk=K_horiz, 
                                   vka=K_vert, ss=Ss,
                                   sy=Sy,storagecoefficient=True, ipakcb=53) 

    rch = flopy.modflow.ModflowRch(m, nrchop=3, rech=2e-5)

    #create oc stress period data. 
    #(0,0) tells OC package to save data for stress period 1, time step 1.
    spd = {(0,0):['print head', 'print budget', 'save head', 'save budget']} #create a dictionary for stress period data, where key (0,0) is associated with value ['print...'] 

    print("oc stress period data: \n", spd)


    oc = flopy.modflow.ModflowOc(model=m, stress_period_data=spd, compact=True)

    pcg = flopy.modflow.ModflowPcg(model=m)

    #Adding a pumping well at the center with flux of 100
    pump_rate = -100
    
    ##LOOK HERE (GROUP 1)
    count = count + 1
    dist = dist_list[count]
    loc = {0:[[0, 10, dist, pump_rate]]}
    print(loc)

    wel = flopy.modflow.mfwel.ModflowWel(m, stress_period_data=loc)  #create object for WEL package


    m.write_input()    #uses the package objects created above to actually write the text files, 
                        #and saves to folder that this script is in
                        #after this step you should see the files appear in your folder

    success, mfoutput = m.run_model(pause=False, report=True)
    if not success:
        raise Exception('MODFLOW did not terminate normally.')



oc stress period data: 
 {(0, 0): ['print head', 'print budget', 'save head', 'save budget']}
{0: [[0, 10, 2, -100]]}
FloPy is using the following  executable to run the model: C:\WRDAPP\MODFLOW\mf2005.exe

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.12.00 2/3/2017                        

 Using NAME file: moldel_2.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2020/02/19 22:34:47

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2020/02/19 22:34:47
 Elapsed run time:  0.021 Seconds

  Normal termination of simulation
oc stress period data: 
 {(0, 0): ['print head', 'print budget', 'save head', 'save budget']}
{0: [[0, 10, 5, -100]]}
FloPy is using the following  executable to run the model: C:\WRDAPP\MODFLOW\mf2005.exe

                                  MODFLOW-2005     
    U.S. GE

In [6]:
count = -1
capture_list = [0,0,0,0,0]
for i in dist_list:
    model = 'moldel' + '_' + str(i)

    budgobj = bf.CellBudgetFile(model+'.cbc')   #reads the cell budget file    
    kstpkper_list = budgobj.get_kstpkper()          #returns a list of stress periods & timesteps
    frf = budgobj.get_data(text='flow right face', totim=1.0) #returns an array of flow rates for right face of cells
    fff = budgobj.get_data(text='flow front face', totim=1.0) #returns an array of flow rates for front face of cells     
             #string options are in the list file under output control (make sure to include spaces!)
    #print("Flow through Right Face of Grid Cells m^3/d \n", frf,
    #     "\n Flow through Front Face of Grid Cells m^3/d \n", fff)

    ##LOOK HERE FOR BUDGET FILE READING
    const_head_flow = budgobj.get_data(text='constant head', totim=1.0)     #extracting the flow in the constant head cells
    chf_array = np.asarray(const_head_flow)      #converting to array
    chf_values = chf_array[0,:]                  #extracting useful component of the array
    chf_list = chf_values.tolist()                   #converting the values to a list
    chf_value_array = np.asarray(chf_list)                            #converting the flow values to array
    chf_cell = chf_value_array [:, 1]                           #separating the flow from the cell number 
    cons_hd_tot_flow = sum(chf_cell)
    const_hd_tot_flow = round(cons_hd_tot_flow,2)
    print('\033[1m' +  "The total capture from the lake is =", const_hd_tot_flow, "m3/day" '\033[0m')
    if const_hd_tot_flow>0:
        per_lake_cap = const_hd_tot_flow*-100/pump_rate 
        print('\033[1m' + "% lake capture =", per_lake_cap,"%" '\033[0m')
    
    count = count + 1
    capture_list[count] = const_hd_tot_flow
    print(capture_list)

The total capture from the lake is = 44.0 m3/day
% lake capture = 44.0 %
[44.0, 0, 0, 0, 0]
The total capture from the lake is = 44.0 m3/day
% lake capture = 44.0 %
[44.0, 44.0, 0, 0, 0]
The total capture from the lake is = 44.0 m3/day
% lake capture = 44.0 %
[44.0, 44.0, 44.0, 0, 0]
The total capture from the lake is = 44.0 m3/day
% lake capture = 44.0 %
[44.0, 44.0, 44.0, 44.0, 0]
The total capture from the lake is = 44.0 m3/day
% lake capture = 44.0 %
[44.0, 44.0, 44.0, 44.0, 44.0]
